絶対に行わなければならないこと<br>
- 糖鎖の残基に対応した原子名の変更
- 糖鎖の残基名の変更
- 残基番号の全 Chain を通しての統一
- 還元末端の探索
- tleap インプットへの出力
- ジスルフィド結合の判定
- 還元末端の水酸基欠損残基に対する水酸基の付与
- PDBの重複構造の除去
- 還元末端の残基に埋もれた O1 原子の並び替えとそれに伴う残基番号の変更
- 糖結合アミノ酸の名前の変更
- ジスルフィド結合する CYS 残基の残基名の CYX への変更

**作業順**<br>
残基番号の修正は極力最後の方が楽<br>
- PDBの重複構造の除去
- 糖鎖の残基に対応した原子名の変更
- 還元末端の探索
- 還元末端の残基に埋もれた O1 原子の並び替えとそれに伴う残基番号の変更
- 還元末端の水酸基欠損残基に対する水酸基の付与
 - 水素原子付加 (subprocess ライブラリを用いた外部プログラムの openbabel の使用)
 - $\alpha$/$\beta$ 体の情報と $C_5 - O_5 - C1 - \{ C_2 , H_{11} , H_{12} \}$ の二面角の並びからどちらの水素を酸素に置換するかの判定
  - RDKIT による情報処理
 - 置換対象原子の原子名の修正
 - 水素原子削除
- PDB 全体の並べ替え
 - タンパク質ブロック、糖鎖ブロック、非糖鎖リガンドブロック、水和水ブロックの順に原子の並べ替えを行う
- 糖鎖の残基名の変更
 - 糖鎖間の結合情報が必要
 - RDKIT による糖鎖間の結合情報の読み取り
- 糖結合アミノ酸の名前の変更
 - 糖鎖間の結合情報が必要
- 残基番号の全 Chain を通しての統一
- ジスルフィド結合の判定
 - PDB ファイルのテキスト情報からの読み取り
 - どのタイミングでも良いが残基番号が修正されたタイミングで元のPDBの番号と修正後の番号との対応が必要
 - 残基番号の修正を極力少なくすることが望ましい
- ジスルフィド結合する CYS 残基の残基名の CYX への変更
- tleap インプットへの出力

In [1]:
# work 1.
import subprocess
import copy
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem import rdDepictor
from rdkit.Chem import rdchem
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import rdMolTransforms,AllChem
from rdkit.Chem import RWMol
from openbabel import openbabel
import sys
import math

In [2]:
# work 2.
inp = "4uo1"
filename = inp + ".pdb"
output0_filename = inp + ".output00.pdb"
output1_filename = inp + ".output01.pdb"
output2_filename = inp + ".output02.pdb"
output3_filename = inp + ".output03.pdb"
output4_filename = inp + ".output04.pdb"
output5_filename = inp + ".output05.pdb"
output6_filename = inp + ".output06.pdb"
output7_filename = inp + ".output07.pdb"
output8_filename = inp + ".output08.pdb"

In [3]:
# work 3.
dict_sugar_code = {\
              "64K":"A","ARA":"a","ARB":"a","LDY":"D","RIP":"R","0MK":"r","XYP":"X","XYS":"X",\
              "LXC":"x","AFD":"N","ALL":"N","WOO":"n","SHD":"E","GAL":"L","GLA":"L","GIV":"l",\
              "GXL":"l","GLC":"G","BGC":"G","GL0":"K","MAN":"M","BMA":"M","BDF":"C","SOE":"b",\
              "T6T":"J","FUC":"f","FUL":"f","RAM":"h","BDP":"Z","GCU":"Z","IDR":"u","NGA":"V",\
              "A2G":"V","NAG":"Y","NDG":"Y","SIA":"S","SLB":"S","ADA":"O","ROH":"ROH",\
              "AMG":"L","MBG":"L","GYP":"G","MMA":"M","MGC":"V","2F8":"Y","MAG":"Y","MNA":"S"
              #"GUP":"g",\
              }
sia_atom_name_mod = {" C10":" C5N"," C11":" CME"," O10":" O5N"}
nacetyl_atom_name_mod = {" C7 ":" C2N"," C8 ":" CME"," O7 ":" O2N"," O  ":" O5 "}

# 2020年7月ごろのPDBの仕様変更により単糖の原子名が修正された模様
# それ以前にDLしたPDBファイルの場合、原子名を以下の辞書を用いて修正する必要があるかも
other_atom_name_mod ={#"64K":{" OAG":" O1 "," CAE":" C1 "," CAD":" C2 "," OAH":" O2 "," CAC":" C3 "," OAI":" O3 "," CAB":" C4 "," OAJ":" O4 "," CAA":" C5 "," OAF":" O5 "},\
                      #"XYP":{" O4A":" O1 "," C1B":" C1 "," C2B":" C2 "," O2B":" O2 "," C3B":" C3 "," O3B":" O3 "," C4B":" C4 "," O4B":" O4 "," C5B":" C5 "," O5B":" O5 "},\
                      #"SHD":{" O6 ":" O1 "," O1 ":" O2 "," O2 ":" O3 "," O3 ":" O4 "," O4 ":" O5 "," O5 ":" O6 "},\
                      #"T6T":{" C1 ":" C6 "," C2 ":" C5 "," O2 ":" O5 "," C3 ":" C4 "," O3 ":" O4 "," C4 ":" C3 "," O4 ":" O3 "," C5 ":" C2 "," O5 ":" O6 "," O5'":" O2 "," C6 ":" C1 "," O6 ":" O1 "}
                     }
o_methyl_sugar_mod = {"MNA":{" C12":" CH3"," O2 ":" O  "},"MAG":{" O1 ":" O  "," CM ":" CH3"},"2F8":{" O1 ":" O  "," CM ":" CH3"},\
                      "MGC":{" O1 ":" O  "," CM ":" CH3"},"MMA":{" O1 ":" O  "," C7 ":" CH3"},"GYP":{" O1 ":" O  "," CM ":" CH3"},\
                      "AMG":{" O1 ":" O  "," CM ":" CH3"},"MBG":{" O1 ":" O  "," CM ":" CH3"}}
list_amino_acid = ['ALA', 'ARG', 'ASN', 'ASP', 'CYS', 'GLU', 'GLN', 'GLY', 'HIS', 'ILE',\
                   'LEU', 'LYS', 'MET', 'PHE', 'PRO', 'SER', 'THR', 'TRP', 'TYR', 'VAL']
list_glycosilation_amino_acid = ["ASN","SER","THR"]
list_sugar_amino_code = [i for i in dict_sugar_code] + list_glycosilation_amino_acid

# 還元末端の水酸基欠損の際の1位水酸基の向き
# L体のFucは現状放置
# work の atom_seq の出力に対応する形に
list_alpha_anomer = ["MAN","NDG","A2G","GLA","GLC",]
list_beta_anomer  = ["BMA","NAG","NGA","GAL","BGC",]

#work#
#枝分かれ糖鎖の残基名(3文字)のうちの最初の1文字のリスト
branch_code_list = ["Z","Y","X","W","V","U","T","S","R","Q","P"]

dict_chiral   = {}
ssbond_pair   = []
dict_anomer_C = {}
list_glycosilation_amino_acid = ("ASN","SER","THR")
list_sugar_amino = [i for i in dict_sugar_code.keys()] + list(list_glycosilation_amino_acid)
dict_connect_list = {\
               frozenset([" C1 "]):"0",frozenset([" C1 "," O2 "]):"2",frozenset([" C1 "," O3 "]):"3",\
               frozenset([" C1 "," O4 "]):"4",frozenset([" C1 "," O6 "]):"6",\
               frozenset([" C1 "," O2 "," O3 "]):"Z",frozenset([" C1 ","O2 "," O4 "]):"Y",\
               frozenset([" C1 "," O2 "," O6 "]):"X",frozenset([" C1 "," O3 "," O4 "]):"W",\
               frozenset([" C1 "," O3 "," O6 "]):"V",frozenset([" C1 "," O4 "," O6 "]):"U",\
               frozenset([" C1 "," O2 "," O3 "," O4 "]):"T",frozenset([" C1 "," O2 "," O3 "," O6 "]):"S",\
               frozenset([" C1 "," O2 "," O4 "," O6 "]):"R",frozenset([" C1 "," O3 "," O4 "," 06 "]):"Q",\
               frozenset([" C1 "," O2 "," O3 "," O4 "," O6 "]):"P"\
              }
dict_connect_list_for_sia = {\
                       frozenset([" C2 "]):"0",frozenset([" C2 "," O4 "]):"4",\
                       frozenset([" C2 "," O7 "]):"7",frozenset([" C2 "," O8 "]):"8",\
                       frozenset([" C2 "," O9 "]):"9",frozenset([" C2 "," O4 "," O7 "," O8 "," O9 "]):"A",\
                       frozenset([" C2 "," O7 "," O8 "," O9 "]):"B",frozenset([" C2 "," O4 "," O8 "," O9 "]):"C",\
                       frozenset([" C2 "," O4 "," O7 ","O9 "]):"D",frozenset([" C2 "," O4 "," O7 "," O8 "]):"E",\
                       frozenset([" C2 "," O8 "," O9 "]):"F",frozenset([" C2 "," O7 "," O9 "]):"G",\
                       frozenset([" C2 "," O7 "," O8 "]):"H",frozenset([" C2 "," O4 "," O9 "]):"I",\
                       frozenset([" C2 "," O4 "," O8 "]):"J",frozenset([" C2 "," O4 "," O7 "]):"K"
                       }
NAG_atom_name_list = {" C7 ":" C2N"," C8 ":" CME"," O7 ":" O2N"}
SIA_atom_name_list = {" C10":" C5N"," C11":" CME"," O10":" O5N"}
#OTH は　Other_sugar　の略
anomer_checker_D = {\
                    ("SIA"," C1 "," O6 "," C3 "):"A",("SIA"," C3 "," C1 "," O6 "):"A",("SIA"," O6 "," C3 "," C1 "):"A",\
                    ("SIA"," C1 "," C3 "," O6 "):"B",("SIA"," C3 "," O6 "," C1 "):"B",("SIA"," O6 "," C1 "," C3 "):"B",\
                    ("OTH"," O5 "," C2 "," H  "):"A",("OTH"," H  "," O5 "," C2 "):"A",("OTH"," C2 "," H  "," O5 "):"A",\
                    ("OTH"," O5 "," H  "," C2 "):"B",("OTH"," H  "," C2 "," O5 "):"B",("OTH"," C2 "," O5 "," H  "):"B",\
                    ("OTH"," O5 "," C2 "," H  "):"A",("OTH"," H  "," O5 "," C2 "):"A",("OTH"," C2 "," H  "," O5 "):"A",\
                    ("OTH"," O5 "," H  "," C2 "):"B",("OTH"," H  "," C2 "," O5 "):"B",("OTH"," C2 "," O5 "," H  "):"B",\
                   }
anomer_checker_L = {\
                    ("SIA"," C1 "," O6 "," C3 "):"B",("SIA"," C3 "," C1 "," O6 "):"B",("SIA"," O6 "," C3 "," C1 "):"B",\
                    ("SIA"," C1 "," C3 "," O6 "):"A",("SIA"," C3 "," O6 "," C1 "):"A",("SIA"," O6 "," C1 "," C3 "):"A",\
                    ("OTH"," O5 "," C2 "," H  "):"B",("OTH"," H  "," O5 "," C2 "):"B",("OTH"," C2 "," H  "," O5 "):"B",\
                    ("OTH"," O5 "," H  "," C2 "):"A",("OTH"," H  "," C2 "," O5 "):"A",("OTH"," C2 "," O5 "," H  "):"A",\
                    ("OTH"," O5 "," C2 "," H  "):"B",("OTH"," H  "," O5 "," C2 "):"B",("OTH"," C2 "," H  "," O5 "):"B",\
                    ("OTH"," O5 "," H  "," C2 "):"A",("OTH"," H  "," C2 "," O5 "):"A",("OTH"," C2 "," O5 "," H  "):"A",\
                   }

In [4]:
len(list_amino_acid)

20

In [5]:
# work 4.A
#PDBの座標情報だけを　～output0.pdb　に出力
#NMR 測定の場合に構造情報が重複している場合があり得る。
#重複情報は MODEL - ENDMDL の繰り返しで確認可能なので最初の1構造のみ取得する
with open(filename) as original:
    with open(output0_filename,"w") as out0:
        for line in original:
            header = line[:6]
            resname = line[17:20]
            if header in ["HETATM" ,"ATOM  "]:
                atomname = line[12:16]
                print ("%s%4s%s" % (line[:12],atomname,line[16:-1]),file=out0)
            if "ENDMDL" in line:
                break

In [6]:
# work 4.B
#座標情報が重複（ASER,BSERのような場合）している場合、A以外を削除して出力
with open(output0_filename) as inp0:
    with open(output1_filename,"w") as out1:
        for line in inp0:
            header = line[:6]
            if header != "CONECT":
                if header in ["HETATM","ATOM  "]:
                    atomname = line[12:16]
                    resname_duplication = line[16:17]
                    if resname_duplication in ["A"," "]:
                        print (line[:-1],file=out1)
                else :
                    print (line[:-1],file=out1)

In [7]:
# work 5.
#原子名の変更、ただしプロトンが付加されているPDBでもプロトンは無視する
#プロトンが存在する場合、原子名を "H" に修正
with open(output1_filename) as inp1:
    with open(output2_filename,"w") as out2:
        for line in inp1:
            if line[:6] in ["HETATM","ATOM  "]:
                serial_num , atomname , resname , chain , resnum = int(line[6:11]) , line[12:16] , line[17:20] , line[21:22] , int(line[22:26])
                if atomname.strip()[0] == "H":
                    new_line = "%11s %4s %3s %1s%4d%s" % ( line[:11] , " H  " , resname , chain , resnum , line[26:-1])
                    print (new_line , file = out2)
                elif resname in ["MNA"]:
                    # 1位水酸基が O-Methyl 化されている場合に、O-methyl基の原子名を変更
                    if atomname in o_methyl_sugar_mod[resname]:
                        new_line = "%11s %4s %3s %1s%4d%s" % ( line[:11] , o_methyl_sugar_mod[resname][atomname],resname,chain,resnum,line[26:-1])
                        print (new_line,file=out2)
                    elif atomname in sia_atom_name_mod:
                        new_line = "%11s %4s %3s %1s%4d%s" % ( line[:11] ,sia_atom_name_mod[atomname],resname,chain,resnum,line[26:-1])
                        print (new_line,file=out2)
                    else:
                        print (line[:-1],file=out2)
                elif resname in ["AMG","MBG","GYP","MMA","MGC","2F8","MAG"]:
                    if atomname in o_methyl_sugar_mod[resname]:
                        new_line = "%11s %4s %3s %1s%4d%s" % ( line[:11] , o_methyl_sugar_mod[resname][atomname],resname,chain,resnum,line[26:-1])
                        print (new_line,file=out2)
                    elif atomname in nacetyl_atom_name_mod:
                        new_line = "%11s %4s %3s %1s%4d%s" % ( line[:11] ,nacetyl_atom_name_mod[atomname],resname,chain,resnum,line[26:-1])
                        print (new_line,file=out2)
                    else:
                        print (line[:-1],file=out2)

                elif resname in ["SIA","SLB"]:
                    #シアル酸の場合、 sia_atom_name_mod を参考に原子名を変更
                    if atomname in sia_atom_name_mod:
                        new_line = "%11s %4s %3s %1s%4d%s" % ( line[:11] ,sia_atom_name_mod[atomname],resname,chain,resnum,line[26:-1])
                        print (new_line,file=out2)
                    else:
                        print (line[:-1],file=out2)
                elif resname in ["NAG","NGA","NDG","A2G"]:
                    #シアル酸以外のNアセチル糖は、 nacetyl_atom_name_mod を参考に原子名を変更
                    if atomname in nacetyl_atom_name_mod:
                        new_line = "%11s %4s %3s %1s%4d%s" % ( line[:11] ,nacetyl_atom_name_mod[atomname],resname,chain,resnum,line[26:-1])
                        print (new_line,file=out2)
                    else:
                        print (line[:-1],file=out2)
                elif resname in other_atom_name_mod:
                    if atomname in other_atom_name_mod[resname]:
                        new_line = "%11s %4s %3s %1s%4d%s" % ( line[:11],other_atom_name_mod[resname][atomname],resname,chain,resnum,line[26:-1])
                        print (new_line,file=out2)
                    else:
                        print(line[:-1],file=out2)
                else:
                    print (line[:-1],file=out2)
            else:
                print (line[:-1],file=out2)

In [8]:
# work 6.A
pdb2 = Chem.MolFromPDBFile(output2_filename,sanitize=False)
dict_O1_miss_attribute = {}
for atom in pdb2.GetAtoms():
    #PDBから全ての原子に対して以下の処理をループ
    resnum = atom.GetPDBResidueInfo().GetResidueNumber()
    resname = atom.GetPDBResidueInfo().GetResidueName()
    atomname = atom.GetPDBResidueInfo().GetName()
    chain = atom.GetPDBResidueInfo().GetChainId()
    if atomname == " O1 ":
        neighbor_atom_list = []
        for neighbor_atom in  atom.GetNeighbors():
            #ある原子に結合しているすべての原子に対してループ処理
            neighbor_atom_resnum = neighbor_atom.GetPDBResidueInfo().GetResidueNumber()
            neighbor_atom_resname = neighbor_atom.GetPDBResidueInfo().GetResidueName()
            neighbor_atom_name = neighbor_atom.GetPDBResidueInfo().GetName()
            neighbor_atom_chain = neighbor_atom.GetPDBResidueInfo().GetChainId()
            neighbor_atom_list.append((neighbor_atom_chain,neighbor_atom_resnum,neighbor_atom_resname,neighbor_atom_name))
        # O1 酸素原子の隣接原子が2つある場合はグリコシド酸素の可能性が高い ※ 水酸基の場合水素原子の情報がPDBにない場合が多いため
        if len(neighbor_atom_list) == 2:
            for i in range(2):
                # 隣接原子の原子名が C1 か H でない (例えば C4 など)場合、酸素原子の残基の帰属と原子名が間違っていることになる
                # 例の場合、 O4 となる
                # この修正を行う
                if neighbor_atom_list[i][3] not in  [" C1 "," H  "]:
                    correct_chain   = neighbor_atom_list[i][0]
                    correct_resnum  = neighbor_atom_list[i][1]
                    correct_resname = neighbor_atom_list[i][2]
                    correct_name = " O" + neighbor_atom_list[i][3][2:]
                    dict_O1_miss_attribute[(chain,resnum,resname,atomname)] \
                    = ( correct_chain,correct_resnum,correct_resname,correct_name)
                
dict_O1_miss_attribute_text = {}
with open(output2_filename) as inp2:
    for line in inp2:
        header = line[:6]
        if header in ["HETATM","ATOM  "]:
            resname    = line[17:20]
            resnum     = int ( line[22:26] )
            chain      = line[21:22]
            serial_num = int(line[6:11])
            atomname   = line[12:16]
            atominfo   = (chain, resnum, resname, atomname)
            if atominfo in dict_O1_miss_attribute:
                correct_chain   = dict_O1_miss_attribute[atominfo][0]
                correct_resnum  = dict_O1_miss_attribute[atominfo][1]
                correct_resname = dict_O1_miss_attribute[atominfo][2]
                correct_name    = dict_O1_miss_attribute[atominfo][3]
                correct_resinfo = (correct_chain, correct_resnum, correct_resname)
                new_line = "%12s%4s %3s %1s%4s%s" % (line[:12], correct_name, correct_resname, correct_chain, correct_resnum, line[26:-1])
                dict_O1_miss_attribute_text[correct_resinfo] = new_line

In [9]:
dict_O1_miss_attribute_text

{}

In [10]:
# work 6.B
# グリコシド酸素の帰属の修正
if len(dict_O1_miss_attribute) != 0:
    with open(output2_filename) as inp3:
        new_serial_num = 0
        old_resinfo    = ""
        with open(output3_filename,"w") as out3:
            for line in inp3:
                if line[:6] in ["HETATM","ATOM  "]:
                    new_serial_num += 1
                    resname    = line[17:20]
                    resnum     = int ( line[22:26] )
                    chain      = line[21:22]
                    serial_num = int(line[6:11])
                    atomname   = line[12:16]
                    atominfo   = (chain, resnum, resname, atomname)
                    resinfo    = (chain, resnum, resname)
                    if (old_resinfo != resinfo)\
                    and ( old_resinfo in [ i[:3] for i in dict_O1_miss_attribute.values()])\
                    and (atomname == " O1 "):
                        new_line = "%s%s%s" % (dict_O1_miss_attribute_text[old_resinfo][:6], new_serial_num, dict_O1_miss_attribute_text[old_resinfo][11:])
                        print(new_line, file = out3)
                    elif (old_resinfo != resinfo)\
                    and ( old_resinfo in [ i[:3] for i in dict_O1_miss_attribute.values()])\
                    and (atomname != " O1 "):
                        new_line = "%s%s%s" % (dict_O1_miss_attribute_text[old_resinfo][:6], new_serial_num, dict_O1_miss_attribute_text[old_resinfo][11:])
                        print(new_line, file = out3)
                        new_serial_num += 1
                        print("%6s%5d%s" % (line[:6],new_serial_num,line[11:-1]) ,file = out3)
                    elif atominfo in dict_O1_miss_attribute:
                        new_serial_num -= 1
                    else:
                        print("%6s%5d%s" % (line[:6],new_serial_num,line[11:-1]) ,file = out3)
                    old_resinfo = resinfo
                else:
                    print(line[:-1] , file = out3)
else:
    subprocess.run("cp %s %s" % (output2_filename,output3_filename))

In [11]:
# work 7.A
# 還元末端の水酸基の欠損を確認するために、プロトン付加する
# 水酸基が欠損している場合、C1 原子にプロトンが2つ付加されることが期待される
run_text = "obabel -h -ipdb %s -opdb -O %s" % (output3_filename,output4_filename)
subprocess.run(run_text)

CompletedProcess(args='obabel -h -ipdb 4uo1.output03.pdb -opdb -O 4uo1.output04.pdb', returncode=0)

In [12]:
# work 7.B
# 確認手続きとして、C1 原子を全てリストアップし辞書に登録、この際の辞書のキーは C1 原子のシリアルナンバー、辞書のアイテムは残基情報
# 続いて付加された水素原子を同じように全てリストアップし辞書に登録
# さらに PDB ファイルの結合情報を全て読み込み辞書化
# この辞書は各原子のシリアルナンバーをキーに、その原子に結合する原子のシリアルナンバーをアイテムに持つ
# 最後にリストアップされた C1 原子に対して結合情報を読み取りその中に水素原子が2つあれば還元末端の水酸基欠損情報としてリストアップ
# また還元末端の水酸基欠損残基の C1 原子周辺の原子情報も収集する
# 次のステップではその残基の C5,O5,C1,C2,H11,H12 のシリアルナンバーを使用するので
# C1 のシリアルナンバーをキーとして、上記の6つの原子のシリアルナンバーをアイテムとする辞書を作成
# ※ シアル酸の場合、 C2 原子に水酸基が結合するため、C1 原子に行う同様の作業を C2 原子に対して行う必要があるが、
#    シアル酸のカルボキシル基が見えていないことは現状想定せず水酸基欠損の探索は行わない
dict_C1_info = {}
dict_O5_info = {}
dict_C5_info = {}
dict_C2_info = {}
dict_H_info  = {}
dict_connect = {}
list_missing_reducing_terminal = []
# PDB のテキスト情報から原子名などを読み取り、後の二面角計算に必要な原子 (C5,O5,C1,C2) を辞書に登録する
with open(output4_filename) as inp4:
    for line in inp4:
        header = line[:6]
        if header in ["HETATM","ATOM  "]:
            resname    = line[17:20]
            resnum     = int ( line[22:26] )
            chain      = line[21:22]
            serial_num = int(line[6:11])
            atomname   = line[12:16]
            resinfo = (chain , resnum , resname)
            atominfo = (chain,resnum,serial_num)
            if resname not in ["SIA","SLB"]:
                if atomname == " C1 ":
                    dict_C1_info[serial_num] = resinfo
                elif atomname == " O5 ":
                    dict_O5_info[serial_num] = resinfo
                elif atomname == " C5 ":
                    dict_C5_info[serial_num] = resinfo
                elif atomname == " C2 ":
                    dict_C2_info[serial_num] = resinfo
                elif atomname == " H  ":
                    dict_H_info[serial_num] = resinfo
#            elif resname in ["SIA","SLB"]:
#                if atomname == " C2 ":
#                    dict_C2_info_for_SIA[serial_num] = resinfo
#                elif atomname == " O5 ":
#                    dict_O5_info[serial_num] = resinfo
#                elif atomname == " C5 ":
#                    dict_C5_info[serial_num] = resinfo
#                elif atomname == " C2 ":
#                    dict_C2_info[serial_num] = resinfo
#                elif atomname == " H  ":
#                    dict_H_info[serial_num] = resinfo
#
#                dict_C2_info_for_SIA[serial_num] = resinfo
#            elif atomname == " H  ":
#                dict_H_info[serial_num] = resinfo
# PDB内の原子数が9999以下の場合、 "CONECT" レコードのシリアルナンバーは空白で分割されるが
# 原子数が10000以上の場合、 CONECT100001000110002 のように数字が連続するので処理を分ける
if serial_num <= 9999:
    with open(output4_filename) as inp4:
        for line in inp4:
            header = line[:6]
            if header == "CONECT":
                serial_num    = int(line.split()[1])
                connect_atoms = [int(i) for i in line.split()[2:]]
                dict_connect[serial_num] = connect_atoms
else:
    with open(output4_filename) as inp4:
        for line in inp4:
            header = line[:6]
            if header == "CONECT":
                connect_atoms = []
                serial_num    = int(line[6:11])
                # 各原子から他の原子への結合数はSP3炭素を想定して最大4として処理
                for connected_atom_num in range(4):
                    connected_atom_serial_num = line[11+5*connected_atom_num:16+5*connected_atom_num]
                    if connected_atom_serial_num != "     ":
                        connect_atoms.append(int(connected_atom_serial_num))
                dict_connect[serial_num] = connect_atoms
    
# C1 原子に隣接する原子を PDB の CONECT レコードから読み取り、4以下の場合はシリアルナンバーを出力                
for C1_serial_num in dict_C1_info:
    flag_missing_reducing_terminal = 0
    if len(dict_connect[C1_serial_num]) != 4:
        print(C1_serial_num)
    # 隣接原子数が4の場合、その中にプロトンがいくつあるか確認、2つある場合は1位水酸基が欠損しているとみなす
    else:
        for C1_connected_atom in dict_connect[C1_serial_num]:
            if C1_connected_atom in dict_H_info:
                flag_missing_reducing_terminal += 1
    if flag_missing_reducing_terminal == 2:
        list_missing_reducing_terminal.append(C1_serial_num)

#for C2_serial_num in dict_C2_info_for_SIA:
#    flag_missing_reducing_terminal_for_SIA = 0
#    if len(dict_connect[C2_serial_num]) != 4:
#        print(C2_serial_num)
#    else:
#        for C2_connected_atom in dict_connect[C2_serial_num]:
#            if C2_connected_atom in dict_H_info:
#                flag_missing_reducing_terminal_for_SIA += 1
#    if flag_missing_reducing_terminal_for_SIA == 2:
#        list_missing_reducing_terminal_for_SIA.append(C2_serial_num)

# 

# 水酸基欠損残基の C1 原子の周辺情報の収集
dict_C1_surround_atom_info = {}
for C1_serial_num in list_missing_reducing_terminal:
    list_C1_surround_atom_info = []
    list_C1_surround_H_info    = []
    for C1_connected_atom in dict_connect[C1_serial_num]:
        if C1_connected_atom in dict_O5_info:
            O5_serial_num = C1_connected_atom
            for O5_connected_atom in dict_connect[O5_serial_num]:
                if O5_connected_atom in dict_C5_info:
                    C5_serial_num = O5_connected_atom
        elif C1_connected_atom in dict_C2_info:
            C2_serial_num = C1_connected_atom
        else:
            list_C1_surround_H_info.append(C1_connected_atom)
    list_C1_surround_atom_info = [C5_serial_num,O5_serial_num,C1_serial_num,C2_serial_num] + list_C1_surround_H_info
    dict_C1_surround_atom_info[C1_serial_num] = list_C1_surround_atom_info

In [13]:
# work 7.C
# 還元末端の水酸基欠損残基が特定できたのでアノマーに合わせて水素原子を酸素原子に置換する
# 2つの水素原子のうちどちらを置換すべきか確定するために
# C5-O5-C1-C2 , C5-O5-C1-H11 , C5-O5-C1-H12 (H11,H12 は C1 に結合した水素原子) の二面角をそれぞれ計算
# 二面角の大きさの順に並べ替えてソートすると D_C2 > D_H11 > D_H12 と　D_C2 > D_H12 > D_H11 の2通りになる
# この組み合わせと残基名からわかる α/β の組み合わせからどちらの水素を酸素原子に置換するか決定する
temp_pdb = Chem.MolFromPDBFile(output4_filename,removeHs=False,sanitize=False)
conf = temp_pdb.GetConformers()[0]
list_O1_substitution = []
for C1_serial_num in dict_C1_surround_atom_info:
    C5_serial_num  = dict_C1_surround_atom_info[C1_serial_num][0]
    O5_serial_num  = dict_C1_surround_atom_info[C1_serial_num][1]
    C2_serial_num  = dict_C1_surround_atom_info[C1_serial_num][3]
    H11_serial_num = dict_C1_surround_atom_info[C1_serial_num][4]
    H12_serial_num = dict_C1_surround_atom_info[C1_serial_num][5]
    # rdMolTransforms.GetDihedralDeg() は二面角を計算する関数、第1引数に構造を、第2-5引数に原子のシリアルナンバーを入力
    # この際に Python の index は 0 から始まる関係で PDB のシリアルナンバーから 1 を引く点に注意
    d_c2  = rdMolTransforms.GetDihedralDeg(conf, C5_serial_num -1, O5_serial_num -1, C1_serial_num -1, C2_serial_num -1)
    d_h11 = rdMolTransforms.GetDihedralDeg(conf, C5_serial_num -1, O5_serial_num -1, C1_serial_num -1, H11_serial_num -1)
    d_h12 = rdMolTransforms.GetDihedralDeg(conf, C5_serial_num -1, O5_serial_num -1, C1_serial_num -1, H12_serial_num -1)
    chiral_check = [(" C2 ",C2_serial_num,d_c2),(" H11",H11_serial_num,d_h11),(" H12",H12_serial_num,d_h12)]
    atom_seq = sorted(chiral_check,key=lambda dihed:dihed[2])
    resname = dict_C1_info[C1_serial_num][2]
    if resname in list_alpha_anomer:
        # α体の場合は 時計回り＝atom_seq C2-O1-H1 の順になる
        if atom_seq[0][0] == " C2 ":
            O1_serial_num = atom_seq[1][1]
        elif atom_seq[1][0] == " C2 ":
            O1_serial_num = atom_seq[2][1]
        elif aotm_seq[2][0] == " C2 ":
            O1_serial_num = atom_seq[0][1]
        list_O1_substitution.append(O1_serial_num)
    else:
        if atom_seq[0][0] == " C2 ":
            O1_serial_num = atom_seq[2][1]
        elif atom_seq[1][0] == " C2 ":
            O1_serial_num = atom_seq[0][1]
        elif atom_seq[2][0] == " C2 ":
            O1_serial_num = atom_seq[1][1]
        list_O1_substitution.append(O1_serial_num)
list_O1_substitution_resinfo = [dict_H_info[i] for i in list_O1_substitution]

In [14]:
# work 7.D
dict_O1_substitution_text = {}
# dict_O1_substitution_text は H 原子を O1 原子に変更する際の H 原子の情報を格納する
# 辞書のキーは resinfo (chain,resnum,resname ) からなるタプル、辞書のアイテムは H 原子の情報のうち原子名のみ O1 に変更した文字列とする
with open(output4_filename) as inp4:
    for line in inp4:
        header = line[:6]
        if header in ["HETATM","ATOM  "]:
            resname    = line[17:20]
            resnum     = int ( line[22:26] )
            chain      = line[21:22]
            serial_num = int(line[6:11])
            atomname   = line[12:16]
            resinfo    = (chain, resnum, resname)
            atominfo   = (chain, resnum, serial_num)
            if serial_num in list_O1_substitution:
                new_line = "HETATM%6s%4s%s%s" % (line[6:12] , " O1 " , line[16:77],"O")
                dict_O1_substitution_text[resinfo] = new_line

In [15]:
# work 7.E
# 水素付加されていない output03 のファイルを読みながら O1 置換残基が存在する場合、残基の最後に
# dict_O1_substitution_test から O1 原子の座標、構造情報を出力する
with open(output3_filename) as inp5:
    with open(output5_filename,"w") as out5:
        old_resinfo = ()
        new_serial_num = 0
        for line in inp5:
            header = line[:6]
            if header in ["HETATM","ATOM  "]:
                new_serial_num += 1
                resname    = line[17:20]
                resnum     = int ( line[22:26] )
                chain      = line[21:22]
                serial_num = int(line[6:11])
                atomname   = line[12:16]
                resinfo = (chain , resnum , resname)
                atominfo = (chain,resnum,serial_num)
                if (old_resinfo != resinfo) and ( old_resinfo in dict_O1_substitution_text):
                    print("%6s%5d%s" % (dict_O1_substitution_text[old_resinfo][:6],new_serial_num,dict_O1_substitution_text[old_resinfo][11:]), file=out5)
                    new_serial_num += 1
                print("%6s%5d%s" % (line[:6],new_serial_num,line[11:-1]) ,file = out5)
                old_resinfo = resinfo
            else:
                print(line[:-1] , file = out5)


In [16]:
# work 8.A
# 原子名から還元末端の水酸基を探索しROHリスト、O-methylリストに登録
roh_list = []
with open(output5_filename) as inp5:
    for line in inp5:
        header = line[:6]
        if header in ["HETATM","ATOM  "]:
            resnum   = int(line[22:26])
            resname  = line[17:20]
            atomname = line[12:16]
            chain    = line[21:22]
            if ( resname in dict_sugar_code and resname not in ["SIA","SLB"] and atomname == " O1 ")\
            or ( resname in ["SIA","SLB"] and atomname == " O2 "):
                roh_list.append([chain,resnum,resname])
och3_list = []
with open(output5_filename) as inp5:
    for line in inp5:
        header = line[:6]
        if header in ["HETATM","ATOM  "]:
            resnum   = int(line[22:26])
            resname  = line[17:20]
            atomname = line[12:16]
            chain    = line[21:22]
            if ( resname in ["MNA","AMG","MBG","GYP","MMA","MGC","2F8","MAG"] and atomname == " O  "):
                och3_list.append([chain,resnum,resname])

In [17]:
# work 8.B
#還元末端の残基の中に埋もれたROH基のO1（シアル酸の場合O2）を行頭に出してずれたシリアル番号の修正
with open(output5_filename) as inp5:
    with open(output6_filename,"w") as out6:
        old_res , delta_resnum = ["",0],0
        for line in inp5:
            if line[:6] in ["HETATM","ATOM  "]:
                serial_num , atomname , resname , chain , resnum = int(line[6:11]) , line[12:16] , line[17:20] , line[21:22] , int(line[22:26])
                new_res = [chain,resnum]
                if new_res != old_res:
                    #現在読み込んでいる行の残基情報と1行前の残基情報を比較して異なっている場合
                    #ROH基のO原子よりも前にある原子数 delta_serial_num を0にセット、還元末端の最初の原子～ROHのO原子までを入れる文字列 new_lines を新たに追加
                    #さらにROH基の位置に関するフラグを設定
                    delta_serial_num ,new_lines , flag = 0 , "" , 0
                    flag_och3 = 0
                if ([ chain , resnum,resname] in roh_list) or ([chain,resnum,resname] in och3_list):
                    #残基の最初はフラグが0、ROH基のOを読み込むとフラグが1となる。
                    #フラグが0の間はシリアル番号が+1（行頭にROH基のO原子が来るため）され、 new_lines に収納
                    #その単糖中のO原子以前の原子数をdelta_serial_numとして記録
                    #ROH基のO原子を読み込むと、残基名をROHにし、O原子のシリアル番号に delta_serial_num を引く
                    #出力ファイルにO原子の行を最初に出力し、その後O原子以前の行を収納している new_lines を書き込む
                    #その後フラグを1に変更
                    #フラグが１になると読み込んだ行をそのまま出力ファイルに書き込む
                    #シアル酸の場合アノマー炭素がC２（アノマー炭素結合水酸基酸素はO2）、他の単糖はアノマー炭素がC1なので別々に処理する
                    if resname in ["SIA","SLB"]:
                        if atomname == " O2 ":
                            new_line0 = "%6s%5d  O1  ROH%s" % ( line[:6] , serial_num -delta_serial_num ,line[20:])
                            new_lines = new_line0 + new_lines
                            print (new_lines[:-1],file=out6)
                            flag = 1
                        elif flag == 0:
                            new_line0 = "%6s%5d%s" % ( line[:6] , serial_num + 1 , line[11:] )
                            new_lines = new_lines + new_line0
                            delta_serial_num += 1
                        elif flag == 1:
                            print (line[:-1],file=out6)
                    elif resname in ["AMG","MBG","GYP","MMA","MGC","2F8","MAG","MNA"]:
                        if atomname in [" O  "," CH3"]:
                            if atomname == " CH3":
                                new_lineC = "%6s%5d %4s OME%s" % ( line[:6] , serial_num -delta_serial_num , line[12:16],line[20:])
                                flag_och3 += 1
                            elif atomname == " O  ":
                                new_lineO = "%6s%5d %4s OME%s" % ( line[:6] , serial_num -delta_serial_num , line[12:16],line[20:])
                                flag_och3 += 1
                            if flag_och3 == 2:
                                Catomnum = new_lineC[6:11]
                                Oatomnum = new_lineO[6:11]
                                if Catomnum > Oatomnum:
                                    new_lineC = "%6s%5s%s" % ( new_lineC[:6] , Oatomnum , new_lineC[11:])
                                    new_lineO = "%6s%5s%s" % ( new_line0[:6] , Catomnum , new_lineO[11:])
                                new_lines = new_lineC + new_lineO + new_lines
                                print(new_lines[:-1],file = out6)
                                new_lines = new_line0 + new_lines
                                flag += 1
                                if flag == 2:
                                    print(new_lines[:-1],file = out6)
                                else:
                                    delta_serial_num += 1
                        elif flag_och3 == 0:
                            new_line0 = "%6s%5d%s" % ( line[:6] , serial_num + 2 , line[11:] )
                            new_lines = new_lines + new_line0
                            delta_serial_num += 1
                        elif flag_och3 == 1:
                            new_line0 = "%6s%5d%s" % ( line[:6] , serial_num + 1 , line[11:] )
                            new_lines = new_lines + new_line0
                            delta_serial_num += 1
                        elif flag_och3 == 2:
                            print (line[:-1],file=out6)
                    elif resname not in  ["SIA","SLB"]:
                        if atomname == " O1 ":
                            new_line0 = "%6s%5d %4s ROH%s" % ( line[:6] , serial_num -delta_serial_num , line[12:16],line[20:])
                            new_lines = new_line0 + new_lines
                            print (new_lines[:-1],file=out6)
                            flag = 1
                        elif flag == 0:
                            new_line0 = "%6s%5d%s" % ( line[:6] , serial_num + 1 , line[11:] )
                            new_lines = new_lines + new_line0
                            delta_serial_num += 1
                        elif flag == 1:
                            print (line[:-1],file=out6)
                    else:
                        print (line[:-1],file=out6)
                else:
                    print (line[:-1],file=out6)
                old_res = new_res

In [18]:
# dict_resbond_ ,dict_connect_resinfo_, list_ssbond_pair_ 関連はここからRUN

In [19]:
# work 9
#ジスルフィド結合に関連するシステインのリストアップ
list_ssbond , list_ssbond_pair = [] , []
with open(filename) as f:
    for line in f:
        header = line[:6]
        if header == "SSBOND":
            cyx1 = ( line.split()[3],int(line.split()[4]),line.split()[2] )
            cyx2 = ( line.split()[6],int(line.split()[7]),line.split()[5] )
            list_ssbond_pair.append([cyx1,cyx2])
            list_ssbond.append(cyx1)
            list_ssbond.append(cyx2)

In [20]:
# work 10.A
# 残基間の結合と結合の際の絶対配座を読むための事前準備としてプロトン付加する
# 水酸基が欠損している場合、C1 原子にプロトンが2つ付加されることが期待される
run_text = "obabel -h -ipdb %s -opdb -O %s" % (output6_filename,output7_filename)
subprocess.run(run_text)

CompletedProcess(args='obabel -h -ipdb 4uo1.output06.pdb -opdb -O 4uo1.output07.pdb', returncode=0)

In [21]:
# work 10.B
# rdkit で解析するために PDB の読み込み
pdb7 = Chem.MolFromPDBFile(output7_filename,removeHs=False,sanitize=False)
conf = pdb7.GetConformers()[0]
pdb7

In [22]:
# work 10.C
#糖鎖と糖鎖結合アミノ酸の残基間結合原子のリストアップ
dict_resbond = {}
for atom in pdb7.GetAtoms():
    resnum   = atom.GetPDBResidueInfo().GetResidueNumber()
    resname  = atom.GetPDBResidueInfo().GetResidueName()
    atomname = atom.GetPDBResidueInfo().GetName()
    chain    = atom.GetPDBResidueInfo().GetChainId()
    resinfo  = (chain, resnum, resname)
    for neighbor_atom in  atom.GetNeighbors():
        neighbor_atom_resnum  = neighbor_atom.GetPDBResidueInfo().GetResidueNumber()
        neighbor_atom_resname = neighbor_atom.GetPDBResidueInfo().GetResidueName()
        neighbor_atom_name    = neighbor_atom.GetPDBResidueInfo().GetName()
        neighbor_atom_chain   = neighbor_atom.GetPDBResidueInfo().GetChainId()
        neighbor_resinfo      = (neighbor_atom_chain, neighbor_atom_resnum, neighbor_atom_resname)
        if resinfo != neighbor_resinfo \
        and ( resname in dict_sugar_code.keys() or neighbor_atom_resname in dict_sugar_code.keys() ):
            #全ての原子に対して隣接原子を読み込み、元の原子と隣接原子の残基番号、残基名が異なり、
            #かつ一方が糖鎖関連の場合辞書に登録
            if resinfo not in dict_resbond:
                dict_resbond[resinfo] = [atomname]
            else:
                dict_resbond[resinfo].append(atomname)
            #辞書のキーは（鎖、残基番号、残基名）のタプル、値は隣接残基と結合している原子名
            #アノマー炭素と非還元末端側のグリコシド酸素が登録される

In [23]:
# work 10.D
#アノマー炭素の情報収集、シアル酸ではC2、それ以外の糖はC1を辞書に登録
for atom in pdb7.GetAtoms():
    resnum   = atom.GetPDBResidueInfo().GetResidueNumber()
    resname  = atom.GetPDBResidueInfo().GetResidueName()
    atomname = atom.GetPDBResidueInfo().GetName()
    chain    = atom.GetPDBResidueInfo().GetChainId()
    idx_pdb  = atom.GetPDBResidueInfo().GetSerialNumber()
    if resname in dict_sugar_code and resname not in  ["SIA","SLB"] and atomname == " C1 ":
        dict_chiral[idx_pdb] = [chain,resnum , resname,atomname]
    if resname in  ["SIA","SLB"] and atomname ==" C2 ":
        dict_chiral[idx_pdb] = [chain,resnum,resname,atomname]
    #辞書のキーはアノマー炭素のシリアル番号、値は（鎖、残基番号、残基名、原子名）のタプル

In [24]:
# work 10.E
#キラリティー辞書に登録したアノマー原子のシリアル番号をキーとして、
#アノマー原子に結合するグリコシドO原子の隣接炭素原子を原子i(ROHの場合はH)
#アノマー原子に結合するグリコシドO原子を原子j
#アノマー原子を原子k
#アノマー原子に結合するグリコシドO原子以外の3原子を原子lと定義した上で
#値に原子i,j,k,lを持つ辞書を作成
#アノマーの定義は本来はアノマー位炭素とアノマー参照炭素（アノマー炭素から最も離れた環内不正炭素、GlcなどではC'4）の絶対配座が
#異なる場合にα、同じ場合βとなる。
#ただしグリコシド結合によりグリコシド酸素に隣接する原子が水素から炭素になることで、絶対配座の原子順位が変わるので単純な絶対配座による比較は不可能。
#その為ここでは下のような3原子の二面角を取り、3原子の順序を比較することでα/βを判定している。
#原子i（隣接残基のグリコシドO原子の隣接炭素原子）-原子j（隣接残基のグリコシドO原子）-原子k（アノマー原子）-原子l（アノマー原子隣接3原子）
#α/βのどちらでも、アノマー炭素に結合している原子ｌは同じ（Glcの場合は、C2,H1,O5）であり、
#α/βでは二面角の大きさで整理した場合の順が変わる
for atom in pdb7.GetAtoms():
    idx_pdb = atom.GetPDBResidueInfo().GetSerialNumber()
    if idx_pdb in dict_chiral:
        #最初に原子k（アノマー炭素）の確定
        resnum   = atom.GetPDBResidueInfo().GetResidueNumber()
        resname  = atom.GetPDBResidueInfo().GetResidueName()
        atomname = atom.GetPDBResidueInfo().GetName()
        chain    = atom.GetPDBResidueInfo().GetChainId()
        resinfo  = (chain, resnum , resname)
        dict_anomer_C[idx_pdb] = []
        dihed_atoml = []
        dihed_atomk_idx  = idx_pdb
        dihed_atomk_name = atomname
        for neighbor_atom in atom.GetNeighbors():
            # アノマー炭素の隣接原子のうち3原子は同じ残基であり、残った1つは隣接残基であるため残基情報が異なる
            neighbor_atom_resnum  = neighbor_atom.GetPDBResidueInfo().GetResidueNumber()
            neighbor_atom_resname = neighbor_atom.GetPDBResidueInfo().GetResidueName()
            neighbor_atom_name    = neighbor_atom.GetPDBResidueInfo().GetName()
            neighbor_atom_chain   = neighbor_atom.GetPDBResidueInfo().GetChainId()
            neighbor_atom_idx     = neighbor_atom.GetPDBResidueInfo().GetSerialNumber()
            neighbor_resinfo      = (neighbor_atom_chain, neighbor_atom_resnum, neighbor_atom_resname)
            if neighbor_resinfo != resinfo:
                #隣接原子のうち残基番号が異なるグリコシド酸素原子場合以下の処理
                #原子jの確定、さらにグリコシド酸素原子の隣接原子（アノマー炭素と反対側の炭素原子）のうち
                #残基番号が同じ原子を原子iと確定
                dihed_atomj_idx  = neighbor_atom_idx
                dihed_atomj_name = neighbor_atom_name
                for neighbor2_atom in neighbor_atom.GetNeighbors():
                    neighbor2_atom_resnum  = neighbor2_atom.GetPDBResidueInfo().GetResidueNumber()
                    neighbor2_atom_resname = neighbor2_atom.GetPDBResidueInfo().GetResidueName()
                    neighbor2_atom_name    = neighbor2_atom.GetPDBResidueInfo().GetName()
                    neighbor2_atom_chain   = neighbor2_atom.GetPDBResidueInfo().GetChainId()
                    neighbor2_atom_idx     = neighbor2_atom.GetPDBResidueInfo().GetSerialNumber()
                    neighbor2_resinfo      = (neighbor2_atom_chain, neighbor2_atom_resnum, neighbor2_atom_resname)
                    
                    if neighbor2_resinfo == neighbor_resinfo:
                        dihed_atomi_idx  = neighbor2_atom_idx
                        dihed_atomi_name = neighbor2_atom_name
            else:
                #隣接原子のうち残基番号が同じ3つの原子を原子lとしてそれぞれ辞書に収録
                dihed_atoml_idx  = neighbor_atom_idx
                dihed_atoml_name = neighbor_atom_name
                dihed_atoml.append([dihed_atoml_name,dihed_atoml_idx])
        dihed_atomijk = [dihed_atomi_name,dihed_atomi_idx,dihed_atomj_name,dihed_atomj_idx,dihed_atomk_name,dihed_atomk_idx]
        for l in dihed_atoml:
            dict_anomer_C[idx_pdb].append( dihed_atomijk + l ) 

In [25]:
# work 10.F
#α/βの判定
for anomer_center in dict_anomer_C:
    chiral_check = []
    for same_res_atom in [0,1,2]:
        # dict_anomer_C には還元末端側グリコシド酸素隣接原子-グリコシド酸素-アノマー炭素にアノマー炭素に結合した3原子が収録されている
        # この2つの二面角を計算し3原子の並びからαかβかの判定を行う
        # 二面角リストの原子 l 3つに対してでループ処理、二面角を求めた後二面角リストのそれぞれのリストに追加収録
        atoml_name = dict_anomer_C[anomer_center][same_res_atom][6]
        # Rdkit.Chem.rdMolTransform.GetDihedralDeg で必要となる原子のシリアルナンバーは
        # Python のインデックスが0から始める関係で PDB ファイル上のシリアルナンバーから 1 を引いた値にする必要がある
        atomi = dict_anomer_C[anomer_center][same_res_atom][1]-1
        atomj = dict_anomer_C[anomer_center][same_res_atom][3]-1
        atomk = dict_anomer_C[anomer_center][same_res_atom][5]-1
        atoml = dict_anomer_C[anomer_center][same_res_atom][7]-1
        chiral_check.append((atoml_name,rdMolTransforms.GetDihedralDeg(conf, atomi,atomj,atomk,atoml)))
    #二面角でソート
    atom_seq = sorted(chiral_check,key=lambda dihed:dihed[1])
    #ソートで得られた原子lの順序からα/βの決定
    atom_first,atom_second,atom_third = atom_seq[0][0],atom_seq[1][0],atom_seq[2][0]
    anomer_center_res = dict_chiral[anomer_center][2]
    #α/βはD型、L型で入れ替わるので判定、Fuc（フコース）やIDR（イズロン酸）はL型が基本なのでリストと照合して判定
    if dict_sugar_code[anomer_center_res].isupper():
        if anomer_center_res in ["SIA","SLB"]:
            anomer_center_res = "SIA"
        else:
            anomer_center_res = "OTH"
        anomer_type = anomer_checker_D[(anomer_center_res,atom_first,atom_second,atom_third)]
    elif dict_sugar_code[anomer_center_res].islower():
        if anomer_center_res in ["SIA","SLB"]:
            anomer_center_res = "SIA"
        else:
            anomer_center_res = "OTH"
        anomer_type = anomer_checker_L[(anomer_center_res,atom_first,atom_second,atom_third)]
    res = tuple ( dict_chiral[anomer_center][0:3] )
    dict_resbond[res].append(anomer_type)

    #dict_resbond辞書にアノマー情報の追加

In [26]:
# work 10.G
#GLYCAM力場では糖鎖は還元末端から非還元末端へ伸長して行くように残基番号を当て嵌める必要があるが
#PDBには異なる順序で登録されている場合があるため、番号の振り直しを行う。
#まず全ての糖鎖のアノマー炭素と結合している残基情報を所得する。
#各行を読みながら、resinfoとして（鎖、残基番号、残基名）のタプルで比較していく。
#辞書dict_connect_resinfo はキーとして還元末端側のresinfoを持ち、値として非還元末端側のresinfoを持つ。
#分岐のある場合、値は2つ以上のresinfoを持つことになる。
dict_connect_resinfo = {}
for atom in pdb7.GetAtoms():
    resnum   = atom.GetPDBResidueInfo().GetResidueNumber()
    resname  = atom.GetPDBResidueInfo().GetResidueName()
    atomname = atom.GetPDBResidueInfo().GetName()
    chain    = atom.GetPDBResidueInfo().GetChainId()
    resinfo  = ( chain,resnum , resname)
    if ( resinfo in dict_resbond ) \
    and (   ( resname not in ["SIA","SLB"] and atomname == " C1 ")\
         or ( resname in      ["SIA","SLB"] and atomname == " C2 ")):
        #アノマー炭素の探索
        for neighbor_atom in atom.GetNeighbors():
            neighbor_atom_resnum  = neighbor_atom.GetPDBResidueInfo().GetResidueNumber()
            neighbor_atom_resname = neighbor_atom.GetPDBResidueInfo().GetResidueName()
            neighbor_atom_name    = neighbor_atom.GetPDBResidueInfo().GetName()
            neighbor_atom_chain   = neighbor_atom.GetPDBResidueInfo().GetChainId()
            neighbor_atom_idx     = neighbor_atom.GetPDBResidueInfo().GetSerialNumber()
            neighbor_resinfo      = (neighbor_atom_chain, neighbor_atom_resnum, neighbor_atom_resname)
            if resinfo != neighbor_resinfo:
                #アノマー炭素とresinfoが異なる＝隣接残基の原子である場合以下の処理
                if neighbor_resinfo not in dict_connect_resinfo:
                    #分岐のある場合還元末端側が同じ残基になることがあり得るので、初めて登録される場合のみリストの追加を行う
                    dict_connect_resinfo[neighbor_resinfo]=[]
                dict_connect_resinfo[neighbor_resinfo].append(resinfo)

In [27]:
# work 10.H
# PDB 全体の並べ替えを実行
# プロトンもここで削除する
# 並べ替える順番は、タンパク質-糖鎖-糖鎖以外のリガンド-水とし
# RDKIT による結合情報を元に並べ替えるので GLYCAM06 力場での使用が想定されていない希少糖、非天然糖を含む
# また Residue Number もこのタイミングで修正する
# Residue Number は Chain ごとに1から開始するように変更する
# この際にジスルフィド結合した CYS 残基の辞書情報と糖鎖結合情報も修正を行う

# 残基番号を Chain ごとに1からの連番に修正
# 同時にSS結合、糖鎖間結合に関連する残基の場合残基番号の変更を記録する
list_sugar_resname = [i[2] for i in dict_resbond.keys()]
dict_protein_block = {}
dict_sugar_block   = {}
dict_ligand_block  = {}
dict_water_block   = {}
dict_chain_resnum  = {}
dict_resinfo_mod   = {}
dict_chain_resinfo_resnum  = {}
dict_delta_resnum_in_chain = {}
with open(output7_filename) as inp7:
    new_serial_num = 0
    for line in inp7:
        header = line[:6]
        if header in ["ATOM  ","HETATM"]:
            new_serial_num += 1
            resname    = line[17:20]
            resnum     = int ( line[22:26] )
            chain      = line[21:22]
            serial_num = int(line[6:11])
            atomname   = line[12:16]
            resinfo = (chain , resnum , resname)
            atominfo = (chain,resnum,serial_num)
            if atomname != " H  ":
                if chain not in dict_chain_resinfo_resnum:
                    dict_chain_resinfo_resnum[chain] = {}
                    dict_delta_resnum_in_chain[chain] = 0
                if resinfo not in dict_chain_resinfo_resnum[chain]:
                    dict_delta_resnum_in_chain[chain] += 1
                    dict_chain_resinfo_resnum[chain][resinfo] = dict_delta_resnum_in_chain[chain]
                    if ( resinfo in dict_resbond ) or ( resinfo in list_ssbond):
                        dict_resinfo_mod[resinfo] = (chain,dict_delta_resnum_in_chain[chain],resname)
                    
                    

# タンパク質、糖鎖、リガンド、水ごとに残基番号の修正を加えながら構造情報を保存
with open(output7_filename) as inp7:
    new_serial_num = 0
    delta_resnum   = 0
    old_resinfo    = ""
    for line in inp7:
        header = line[:6]
        if header in ["ATOM  ","HETATM"]:
            new_serial_num += 1
            resname    = line[17:20]
            resnum     = int ( line[22:26] )
            chain      = line[21:22]
            serial_num = int(line[6:11])
            atomname   = line[12:16]
            resinfo = (chain , resnum , resname)
            atominfo = (chain,resnum,serial_num)
            if atomname != " H  ":
                if resinfo != old_resinfo and resinfo[0] == resinfo[0]:
                    delta_resnum += 1
                new_serial_num += 1
                new_resnum = dict_chain_resinfo_resnum[chain][resinfo]
                new_line = "%6s%5d%s%4d%s" % (line[:6], new_serial_num, line[11:22], new_resnum, line[26:-1])
                old_resinfo = resinfo
                if resname in list_amino_acid:
                    dict_protein_block[new_serial_num] = new_line
                elif resname in list_sugar_resname:
                    dict_sugar_block[new_serial_num]   = new_line
                elif resname not in ["WAT","HOH"]:
                    dict_ligand_block[new_serial_num]  = new_line
                elif resname in ["WAT","HOH"]:
                    dict_water_block[new_serial_num]   = new_line
                else:
                    print("Non definition residue: %s" % resname)

# 保存した構造情報を新しいファイルに出力
with open(output8_filename,"w") as out8:
    for block in [dict_protein_block,dict_sugar_block,dict_ligand_block,dict_water_block]:
        for serial_num in block:
            print(block[serial_num] , file = out8)

**詳細**<br>
1. ライブラリの読み込み
1. ファイル名の設定
1. 文字列処理などに必要な辞書、リストの準備
1. PDB ファイルから重複原子の構造情報の除去と測定者、結合情報などの削除
 1. MDL-ENDMDL で囲われた重複構造のうち最初の1構造を出力、同時に著者、結合情報などの原子の構造情報以外を削除
 1. Alternate location識別子 が記述されている場合、 A の構造情報のみ出力
1. 糖鎖の残基に対応した原子名の変更
1. グリコシド結合の酸素原子は本来還元末端側の残基に帰属する。PDB のグリコシド酸素の帰属が正しいかチェック
 1. 原子名が O1 となっている酸素の帰属が正しいか、間違っていれば正しい残基番号、残基名、原子名をO1帰属修正用辞書に登録
 1. O1帰属修正用辞書の情報を元に原子名を修正
 
1. 還元末端の水酸基欠損残基に対する水酸基の付与
 1. 水素原子付加 (subprocess ライブラリを用いた外部プログラムの openbabel の使用)
 1. 2つのプロトンが C1 原子に隣接しているかどうかで1位水酸基の欠損を判定、同時に欠損の場合に後の処理で必要な周辺の原子情報の収集
 1. $\alpha$/$\beta$ 体の情報と $C_5 - O_5 - C1 - \{ C_2 , H_{11} , H_{12} \}$ の二面角の並びからどちらの水素を酸素に置換するかの判定
 1. 酸素原子に置換する水素原子が決定したら原子名を " O1 " に修正し、修正用の辞書に登録<br>
 この際に openbabel で付加した水素原子は座標レコードの最後に来るため、シリアルナンバーの修正も必要
 1. 水素付加前の PDB ファイル ( inp.output03.pdb ) から座標情報を読みながら1位水酸基欠損の残基が来た場合その残基の最後に<br>
 O1 原子の座標情報を加える。この際に原子のシリアルナンバーは全て振り直す<br>
 ※ この段階ではシリアルナンバーをキーとする情報はないので特に影響はない
 - RDKIT による情報処理
1. 還元末端の探索
 1. 原子名から O1 であれば還元末端の水酸基として、<br>
 また O-Methyl化された残基のうち原子名が " O  " であれば還元末端の O-Methyl 基として辞書に登録する
 1. 還元末端の残基の中に埋もれたROH基のO1（シアル酸の場合O2）を行頭に出してずれたシリアル番号の修正
1. 次の残基並べ替えの前にジスルフィド結合をしているシステイン残基ペアをオリジナルの PDB ファイルからテキスト処理で抽出
1. PDB 全体の並べ替え
 1. 糖鎖間の結合情報を得るためにRDKITを用いて絶対配座や残基間結合情報を得る、その事前準備としてプロトン付加を実行
 1. プロトン付加した構造を rdkit.Chem を用いて読み込む
 1. 隣接原子間の残基情報（Chain、残基番号、残基名）を読み取り異なっている場合残基間結合と定義する。<br>
 この際にどちらかの原子が糖鎖と定義されていれば結合辞書に登録する<br>
 またこの辞書は残基情報をキーとして、隣接残基と結合している原子名を values として持つ。<br>
 この辞書に登録されるのは、糖鎖と結合しているアミノ酸および単糖となる。<br>
 そのため、 values にはアミノ酸の場合{ND2,OG,OG1}が、単糖の場合 C1 を必ず含み非還元末端側の水酸基を複数含む可能性がある。
 1. アノマー炭素の情報収集、シアル酸ではC2、それ以外の糖はC1を辞書に登録
 1. $\alpha / \beta$ の判定に必要なアノマー炭素周辺の原子情報の収集
 1. アノマー炭素周辺の原子の二面角を計算し、 PDB ファイルの残基名および二面角の並びから<br>
 $\alpha / \beta$ の判定を行い結合辞書に登録
 1. 糖鎖の C1 原子（シアル酸の場合 C2 原子）と結合している隣接残基情報を収集し糖鎖間結合情報を辞書に登録
 1. タンパク質ブロック、糖鎖ブロック、非糖鎖リガンドブロック、水和水ブロックの順に原子の並べ替えを行う
1. これまでに作成した残基情報辞書に対して 10.G の操作での変更を反映させる

. ROH,OCH3,糖鎖結合アミノ酸と結合している糖鎖を読み込み辞書に登録
 
- 糖鎖の残基名の変更
 - 糖鎖間の結合情報が必要
 - RDKIT による糖鎖間の結合情報の読み取り
- 糖結合アミノ酸の名前の変更
 - 糖鎖間の結合情報が必要
- 残基番号の全 Chain を通しての統一
- ジスルフィド結合の判定
 - PDB ファイルのテキスト情報からの読み取り
 - どのタイミングでも良いが残基番号が修正されたタイミングで元のPDBの番号と修正後の番号との対応が必要
 - 残基番号の修正を極力少なくすることが望ましい
- ジスルフィド結合する CYS 残基の残基名の CYX への変更
- tleap インプットへの出力

In [28]:
# work 11
# 残基修正辞書の情報を元に、糖鎖結合原子辞書、糖鎖関連結合辞書、ジスルフィド結合辞書の修正

dict_resbond_         = {dict_resinfo_mod[i] : dict_resbond[i] for i in dict_resbond.keys()}
dict_connect_resinfo_ = {dict_resinfo_mod[i] : [dict_resinfo_mod[j] for j in dict_connect_resinfo[i]] for i in dict_connect_resinfo.keys()}
list_ssbond_pair_     = [[dict_resinfo_mod[i] for i in j] for j in list_ssbond_pair]
dict_resbond          = copy.deepcopy(dict_resbond_)
dict_connect_resinfo  = copy.deepcopy(dict_connect_resinfo_)
list_ssbond_pair      = copy.deepcopy(list_ssbond_pair_)

In [29]:
# work 12.A
# 糖鎖の残基名を修正するために還元末端側である {"ROH"/"ASN"/"SER"/"THR"/"OME"} の残基から
# 非還元末端側の糖鎖を順次読み進めていきながら辞書を作成
# この際に ChainA の ASN 残基から続く糖鎖は ChainA に変更するなどの修正を加える

dict_glycan_first_resnum = {i:1 for i in dict_chain_resinfo_resnum.keys()}

#work9#
dict_connect_resinfo_temp = copy.deepcopy(dict_connect_resinfo)
dict_glycan_residue_mod = {}
for chain in dict_chain_resinfo_resnum.keys():
    flag_reducing_terminal = 0
    #鎖ごとに糖鎖配列の並べ替え
    list_reducing_terminal , branch_sugar , i  = [] , [] , 0
    #鎖ごとに分岐糖鎖用の配列の用意
    #resifoは（"Chain名","残基番号","残基名"）のタプル
    for resinfo in dict_connect_resinfo_temp:
        if resinfo[0] == chain and resinfo[2] in ["ROH","ASN","SER","THR","OME"]:
            #還元末端の探索
            list_reducing_terminal.append(resinfo)
            flag_reducing_terminal = 1
    if flag_reducing_terminal == 1:
        reducing_side_res = list_reducing_terminal[0]
        del list_reducing_terminal[0]
        while reducing_side_res in dict_connect_resinfo_temp:
            if len(dict_connect_resinfo_temp[reducing_side_res]) >=2:
                #分岐が存在する場合
                branch_sugar.append(reducing_side_res)
                #分岐元の残基を分岐配列に挿入
                if reducing_side_res not in dict_glycan_residue_mod:
                    #分岐鎖の場合、最初の探索の際にresinfoと修正後残基番号の登録を行い、
                    #一方の糖鎖の終端で探索が終わり、再度分岐鎖からもう一方の糖鎖の探索を開始する。
                    #この際に2度目以降の探索で残基番号の修正は行われないようにする
                    #分岐が3つ以上の場合2つ目の分岐でこの判定が有効になる
                    dict_glycan_residue_mod[reducing_side_res] = []
                    dict_glycan_residue_mod[reducing_side_res].append(chain)
                    dict_glycan_residue_mod[reducing_side_res].append(i + dict_glycan_first_resnum[chain])
                    i += 1
                next_reducing_side_res = dict_connect_resinfo_temp[reducing_side_res][0]
                #還元末端側残基をスライドさせる
                del dict_connect_resinfo_temp[reducing_side_res][0]
                #既に読み終わった残基のうち読み込んだ分岐側のresinfoを消去
                reducing_side_res = next_reducing_side_res
            elif len(dict_connect_resinfo_temp[reducing_side_res]) == 1:
                #分岐が存在しない場合
                if reducing_side_res not in dict_glycan_residue_mod:
                    #2分岐糖鎖の2分岐目の際に新たに残基番号の修正を行わないように処理を限定
                    if reducing_side_res[2] in [ "ASN","SER","THR"]:
                        dict_glycan_residue_mod[reducing_side_res] = []
                        dict_glycan_residue_mod[reducing_side_res].append(chain)
                        dict_glycan_residue_mod[reducing_side_res].append( reducing_side_res[1])
                    else:
                        dict_glycan_residue_mod[reducing_side_res] = []
                        dict_glycan_residue_mod[reducing_side_res].append(chain)
                        dict_glycan_residue_mod[reducing_side_res].append( i + dict_glycan_first_resnum[chain])
                        i += 1
                next_reducing_side_res = dict_connect_resinfo_temp[reducing_side_res][0]
                del dict_connect_resinfo_temp[reducing_side_res][0]
                reducing_side_res = next_reducing_side_res
            elif len(dict_connect_resinfo_temp[reducing_side_res]) == 0:
                print("Loop Error")
                break

            if reducing_side_res not in dict_connect_resinfo_temp and len(branch_sugar) >= 1:
                #還元末端から非還元末端へ残基を辿り、終端まで来た場合、
                #終端になった残基の残基番号の修正情報を収録する
                #分岐鎖配列に残基が登録されていればそこから再開始
                dict_glycan_residue_mod[reducing_side_res] = []
                dict_glycan_residue_mod[reducing_side_res].append(chain)
                dict_glycan_residue_mod[reducing_side_res].append( i + dict_glycan_first_resnum[chain])
                reducing_side_res = branch_sugar[0]
                i += 1
                del branch_sugar[0]
            elif reducing_side_res not in dict_connect_resinfo_temp and len(branch_sugar) == 0:
                #還元末端から非還元末端へ残基を辿り、終端まで来た場合、分岐鎖配列に残基が登録されていなければ
                #以下の処理、さらに還元末端リストに別の還元末端が存在するかどうかの判定へ進む
                dict_glycan_residue_mod[reducing_side_res] = []
                dict_glycan_residue_mod[reducing_side_res].append(chain)
                dict_glycan_residue_mod[reducing_side_res].append(i + dict_glycan_first_resnum[chain])
                i += 1
                if len(list_reducing_terminal) != 0:
                    reducing_side_res = list_reducing_terminal[0]
                    del list_reducing_terminal[0]

In [30]:
dict_glycan_residue_mod

{('A', 62, 'ASN'): ['A', 62],
 ('G', 1, 'NAG'): ['A', 1],
 ('G', 2, 'NAG'): ['A', 2],
 ('A', 164, 'ASN'): ['A', 164],
 ('H', 1, 'NAG'): ['A', 3],
 ('H', 2, 'NAG'): ['A', 4],
 ('H', 3, 'BMA'): ['A', 5],
 ('H', 4, 'MAN'): ['A', 6],
 ('H', 5, 'MAN'): ['A', 7],
 ('A', 284, 'ASN'): ['A', 284],
 ('I', 1, 'NAG'): ['A', 8],
 ('I', 2, 'NAG'): ['A', 9],
 ('A', 7, 'ASN'): ['A', 7],
 ('J', 1, 'NAG'): ['A', 10],
 ('J', 2, 'NAG'): ['A', 11],
 ('J', 3, 'BMA'): ['A', 12],
 ('J', 4, 'FUC'): ['A', 13],
 ('A', 37, 'ASN'): ['A', 37],
 ('K', 1, 'NAG'): ['A', 14],
 ('K', 2, 'NAG'): ['A', 15],
 ('K', 3, 'BMA'): ['A', 16],
 ('A', 52, 'ASN'): ['A', 52],
 ('A', 326, 'NAG'): ['A', 17],
 ('B', 154, 'ASN'): ['B', 154],
 ('M', 1, 'NAG'): ['B', 1],
 ('M', 2, 'FUC'): ['B', 2],
 ('C', 47, 'ASN'): ['C', 47],
 ('N', 1, 'NAG'): ['C', 1],
 ('N', 2, 'NAG'): ['C', 2],
 ('C', 159, 'ASN'): ['C', 159],
 ('O', 1, 'NAG'): ['C', 3],
 ('O', 2, 'NAG'): ['C', 4],
 ('O', 3, 'BMA'): ['C', 5],
 ('O', 4, 'MAN'): ['C', 6],
 ('O', 5, 'MAN

In [ ]:
dict_resbond

In [ ]:
#work10-11#
list_glycan_Reduce_term = ["NLN","OLS","OLT","ROH","OME"]
for i in dict_resbond:
    #糖鎖結合アミノ酸（Asn、Ser、Thr）、還元末端ROH、糖鎖の残基名修正辞書への修正情報の収録
    if i[2] == "ASN":
        dict_glycan_residue_mod[i].append("NLN")
    elif i[2] == "SER":
        dict_glycan_residue_mod[i].append("OLS")
    elif i[2] == "THR":
        dict_glycan_residue_mod[i].append("OLT")
    else:
        if i[2] == "ROH":
            dict_glycan_residue_mod[i].append("ROH")
        elif i[2] == "OME":
            dict_glycan_residue_mod[i].append("OME")
        else:
            if i[2] in ["SIA","SLB"]:
                if frozenset(dict_resbond[i][:-1]) in dict_connect_list_for_sia:
                    connect = dict_connect_list_for_sia[frozenset(dict_resbond[i][:-1])]
                else:
                    print("Non Defined Connection was found: %s" % str(i))
            else:
                if frozenset(dict_resbond[i][:-1]) in dict_connect_list:
                    connect = dict_connect_list[frozenset(dict_resbond[i][:-1])]
                else:
                    print("Non Defined Connection was found: %s" % str(i))
            glycam_restype_name = dict_sugar_code[i[2]]
            anomer_type = dict_resbond[i][-1]
            new_resname = connect + glycam_restype_name + anomer_type
            dict_glycan_residue_mod[i].append( new_resname )
            if new_resname not in list_glycan_Reduce_term:
                list_glycan_Reduce_term.append(new_resname)

In [ ]:
#work13-14#
#糖鎖の残基名と残基番号の修正
#ジスルフィドシステインの残基名の修正
with open(output8_filename) as inp9:
    with open(output9_filename,"w") as output9:
        for line in inp9:
            resname    = line[17:20]
            resnum     = int ( line[22:26] )
            chain      = line[21:22]
            serial_num = line[6:11]
            target_residue = ( chain , resnum,resname)
            if target_residue in glycan_residue_mod:
                new_resname = glycan_residue_mod[target_residue][2]
                new_resnum  = glycan_residue_mod[target_residue][1]
                new_line = "%s%s%s%4d%s" % ( line[:17] , new_resname , line[20:22] , new_resnum , line[26:-1] )
                print (new_line,file=output9)
            elif target_residue in cys_mod_list1:
                new_resname = "CYX"
                new_line = "%s%s%s" % ( line[:17] , new_resname , line[20:-1] )
                print (new_line,file=output9)
            else:
                print (line[:-1],file=output9)

In [ ]:
#work8#
#鎖一覧と、各鎖に含まれる糖鎖の開始番号の決定
list_chian_resnum = []
for i in resbond:
    list_chian_resnum.append(i[0])
list_chian_resnum_uniq = list(set(list_chian_resnum))
list_chian_resnum_uniq
chain_total_num = len(list_chian_resnum_uniq)
first_resnum_glycan={}
resnum_array = []
for chain in list_chian_resnum_uniq:
    for ligand_resnum in resbond:
        #糖鎖（糖鎖結合アミノ酸は除く）の残基番号をリスト化し、その最小値を開始番号とする
        if ligand_resnum[2] not in ["ASN","SER","THR"]:
            resnum_array.append(ligand_resnum[1])
    first_resnum_glycan[chain] = min(resnum_array)